
# Credit Scorecard Building Steps:
1. ### Feature Selection:
   1. Top 100 features from  1. Xgb 2. Recursive Feature selection 3. Wrapper Method 4. Filter Method 5. Embedded method 6.Elastic Net Selection
   2. Stable features to consider basis VSI
   3. Select Top 30 Features by looking at their intutive trend with respect to target
2. ### Modelling:
   1. Logistic Regression:
       1. Variable Discretization by WOE and Highest IV variables
       2. Using woe directly
       3. Using dummy variables
          
   2. Xgboost Model :
       1. Raw variables /convert categorical to Dummy variables
3. ### 3 Digit Score 
    1. Probability to 3 digit Score conversion
    2. Attribute wise score
       
4. ### Performance
   1. KS GINI ROC & Gain Table
   2. Optimal cutoff

5. ### Model Explainability
   1. Bivariate plot
   2. Shaply values
  
7. ### Model Monitoring
    1. PSI
    2. CSI
    3. Early Risk Metrics Bad rate trend
  
8. ### Model Deployemnt Flask Application











In [1]:
#load required liabraries 
import pandas as pd
import numpy as np

##few custom settings
pd.options.display.max_columns=None
pd.options.display.max_rows=None

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.drawing.image import Image
import os

In [32]:
loan_data_backup = pd.read_csv('loan_data_2007_2014.csv')

/var/folders/nb/r3vglnfj14v41b4lyl2zgxbh0000gn/T/ipykernel_8827/2954675577.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data_backup = pd.read_csv('loan_data_2007_2014.csv')


In [5]:
loan_data = loan_data_backup.copy()

In [6]:
loan_data.shape

(466285, 75)

In [ ]:
#loan_data.groupby(['id'],dropna=False)['id'].count().sort_values(ascending=True).head()

In [ ]:
loan_data.head()

In [ ]:
loan_data.columns.values

In [ ]:
loan_data.info()

In [ ]:
loan_data['emp_length'].unique()
# Displays unique values of a column.

In [ ]:
loan_data.groupby(['emp_length'],dropna=False)['id'].count()

In [33]:
#***
loan_data['emp_length_int'] = loan_data['emp_length'].astype(str).str.replace('+ years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].astype(str).str.replace('< 1 year', str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].astype(str).str.replace(' years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].astype(str).str.replace(' year', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].astype(str).str.replace("nan",  str('-999'))
# We store the preprocessed ‘employment length’ variable in a new variable called ‘employment length int’,
# We assign the new ‘employment length int’ to be equal to the ‘employment length’ variable with the string ‘+ years’
# replaced with nothing. Next, we replace the whole string ‘less than 1 year’ with the string ‘0’.
# Then, we replace the ‘n/a’ string with the string ‘0’. Then, we replace the string ‘space years’ with nothing.
# Finally, we replace the string ‘space year’ with nothing.

In [ ]:
loan_data.loc[:,['emp_length','emp_length_int']][(loan_data['emp_length'])=='10+ years'].head()

In [ ]:
loan_data.loc[:,['emp_length','emp_length_int']][pd.isna(loan_data['emp_length'])].head()

In [ ]:
loan_data.groupby(['emp_length_int'],dropna=False)['id'].count()

In [ ]:
type(loan_data['emp_length_int'][0])
# Checks the datatype of a single element of a column.

In [34]:
loan_data['emp_length_int'] = pd.to_numeric(loan_data['emp_length_int'])
# Transforms the values to numeric.

In [ ]:
type(loan_data['earliest_cr_line'][0])
# Displays a column.

In [ ]:
loan_data['earliest_cr_line'].head()

In [35]:
#***
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'], format = '%d-%m-%Y')
# Extracts the date and the time from a string variable that is in a given format.

In [ ]:
loan_data.loc[:,['earliest_cr_line','earliest_cr_line_date']][loan_data['earliest_cr_line_date']=='2001-12-01'].head()

In [ ]:
loan_data.groupby(['earliest_cr_line_date'],dropna=False)['id'].count().head()

In [17]:
#loan_data['issue_d']

In [36]:
#***
loan_data['issue_date'] = pd.to_datetime(loan_data['issue_d'], format = '%d-%m-%Y')
# Extracts the date and the time from a string variable that is in a given format.

In [ ]:
loan_data.loc[:,['issue_d','issue_date']][loan_data['issue_date']=='2009-12-01'].head(10)

In [ ]:
type(loan_data['issue_date'][0])

In [37]:
#****
# Assume we are now in December 2017
loan_data['mths_since_earliest_cr_line'] = (loan_data['issue_date'].dt.year - loan_data['earliest_cr_line_date'].dt.year) * 12 + loan_data['issue_date'].dt.month - loan_data['earliest_cr_line_date'].dt.month
# We calculate the difference between two dates in months, turn it to numeric datatype and round it.
# We save the result in a new variable.

In [ ]:
loan_data['mths_since_earliest_cr_line'].describe()

In [26]:
#loan_data.loc[:,['earliest_cr_line_date','issue_date','mths_since_earliest_cr_line','mths_since_earliest_cr_line_']][loan_data['issue_date']=='2009-12-01'].head(10)

In [ ]:
loan_data['term'].unique()

In [38]:
loan_data['term_int'] = pd.to_numeric(loan_data['term'].str.replace(' months', ''))

In [ ]:
loan_data.groupby(['term_int'],dropna=False)['id'].count()

### Preprocessing few discrete variables

In [13]:
loan_data_dummies = [pd.get_dummies(loan_data['grade'], prefix = 'grade', prefix_sep = ':'),
                     pd.get_dummies(loan_data['sub_grade'], prefix = 'sub_grade', prefix_sep = ':'),
                     pd.get_dummies(loan_data['home_ownership'], prefix = 'home_ownership', prefix_sep = ':'),
                     pd.get_dummies(loan_data['verification_status'], prefix = 'verification_status', prefix_sep = ':'),
                     pd.get_dummies(loan_data['loan_status'], prefix = 'loan_status', prefix_sep = ':'),
                     pd.get_dummies(loan_data['purpose'], prefix = 'purpose', prefix_sep = ':'),
                     pd.get_dummies(loan_data['addr_state'], prefix = 'addr_state', prefix_sep = ':'),
                     pd.get_dummies(loan_data['initial_list_status'], prefix = 'initial_list_status', prefix_sep = ':')]
# We create dummy variables from all 8 original independent variables, and save them into a list.
# Note that we are using a particular naming convention for all variables: original variable name, colon, category name.

In [14]:
loan_data_dummies = pd.concat(loan_data_dummies, axis = 1).astype(int)
# We concatenate the dummy variables and this turns them into a dataframe.

In [ ]:
loan_data_dummies.head()

In [15]:
loan_data = pd.concat([loan_data, loan_data_dummies], axis = 1)
# Concatenates two dataframes.
# Here we concatenate the dataframe with original data with the dataframe with dummy variables, along the columns. 

In [ ]:
loan_data.columns.values
# Displays all column names.

### Check for missing values and clean

In [ ]:
loan_data.isnull().sum()
# It returns 'False' if a value is not missing and 'True' if a value is missing, for each value in a dataframe.

In [39]:
# 'Total revolving high credit/ credit limit', so it makes sense that the missing values are equal to funded_amnt.
loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'], inplace=True)
# We fill the missing values with the values of another variable.

/var/folders/nb/r3vglnfj14v41b4lyl2zgxbh0000gn/T/ipykernel_8827/1984499578.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  loan_data['total_rev_hi_lim'].fillna(loan_data['funded_amnt'], inplace=True)


In [ ]:
loan_data['total_rev_hi_lim'].isnull().sum()

In [40]:
def ratio_variable(x,y):
    if pd.isnull(x):
        return None
    elif pd.isnull(y) or y==0:
        return None
    else:
        return round(x/y,2)

In [41]:
# Using apply with a lambda function
loan_data['live_accts_ratio'] = loan_data.apply(lambda row: ratio_variable(row['open_acc'], row['total_acc']), axis=1)


In [ ]:
#loan_data[loan_data['total_acc']==4][['live_accts_ratio','open_acc','total_acc']].head()

In [42]:
predictors=['grade','sub_grade','emp_length_int','home_ownership','purpose','addr_state','dti','delinq_2yrs','inq_last_6mths',
            'mths_since_last_delinq','mths_since_last_record','pub_rec','revol_util','emp_length_int','live_accts_ratio',
            'mths_since_earliest_cr_line']

In [ ]:
loan_data[predictors].isnull().sum()

In [43]:
loan_data['mths_since_earliest_cr_line'].fillna(-999, inplace=True)
loan_data['acc_now_delinq'].fillna(-999, inplace=True)
loan_data['total_acc'].fillna(-999, inplace=True)
loan_data['pub_rec'].fillna(-999, inplace=True)
loan_data['open_acc'].fillna(-999, inplace=True)
loan_data['inq_last_6mths'].fillna(-999, inplace=True)
loan_data['delinq_2yrs'].fillna(-999, inplace=True)
loan_data['emp_length_int'].fillna(-999, inplace=True)
loan_data['live_accts_ratio'].fillna(-999, inplace=True)
loan_data['mths_since_last_delinq'].fillna(-999, inplace=True)
loan_data['mths_since_last_record'].fillna(-999, inplace=True)
loan_data['revol_util'].fillna(-999, inplace=True)

# We fill the missing values with zeroes.
#loan_data['annual_inc'].fillna(-999, inplace=True)
# We fill the missing values with the mean value of the non-missing values.

/var/folders/nb/r3vglnfj14v41b4lyl2zgxbh0000gn/T/ipykernel_8827/3110077339.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  loan_data['mths_since_earliest_cr_line'].fillna(-999, inplace=True)
/var/folders/nb/r3vglnfj14v41b4lyl2zgxbh0000gn/T/ipykernel_8827/3110077339.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

In [ ]:
loan_data[predictors].isnull().sum()

In [ ]:
# Check data types of columns using dtypes
column_types = loan_data[predictors].dtypes

# Display the data types
print(column_types)

## Target Definition Creation - Good/Bad(Default) Account

In [ ]:
loan_data['loan_status'].unique()
# Displays unique values of a column.

In [ ]:
loan_data['loan_status'].value_counts()
# Calculates the number of observations for each unique value of a variable.

In [ ]:
loan_data['loan_status'].value_counts() / loan_data['loan_status'].count()
# We divide the number of observations for each unique value of a variable by the total number of observations.
# Thus, we get the proportion of observations for each unique value of a variable.

In [44]:
# Good/ Bad Definition
loan_data['good_bad'] = np.where(loan_data['loan_status'].isin(['Charged Off', 'Default',
                                                       'Does not meet the credit policy. Status:Charged Off',
                                                       'Late (31-120 days)']), 0, 1)
# We create a new variable that has the value of '0' if a condition is met, and the value of '1' if it is not met.

In [ ]:
loan_data.groupby(['good_bad'],dropna=False)['id'].count()

## Assigning sample weight

In [45]:
loan_data["wgt"]=1

In [ ]:
loan_data.columns

In [50]:
#loan_data.to_csv('loan_data_processed.csv', index=False)

In [ ]:
#loan_data=pd.read_csv('loan_data_processed.csv')

/var/folders/nb/r3vglnfj14v41b4lyl2zgxbh0000gn/T/ipykernel_8827/1790718232.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data=pd.read_csv('loan_data_processed.csv')


### Creating Dev (Train+Test) & OOT data Set

In [46]:
dev = loan_data[loan_data['issue_date'].dt.year != 2014]
oot = loan_data[loan_data['issue_date'].dt.year == 2014]


In [47]:
X=dev.drop('good_bad', axis = 1)
y=dev[['good_bad']]

In [49]:
# Imports the libraries we need.
from sklearn.model_selection import train_test_split

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                                    test_size=0.2,random_state=42, stratify=y)

In [52]:
X_oot,y_oot=oot.drop('good_bad', axis = 1),oot[['good_bad']]

In [53]:
y_train.groupby('good_bad')['good_bad'].count()

good_bad
0     25222
1    159303
Name: good_bad, dtype: int64

In [64]:
summary= dev.agg({'good_bad': ['count', 'sum']})


In [54]:
### Card Data Creation
X_train['ind']='train'
train=pd.concat([X_train, y_train], axis=1)
X_test['ind']='test'
test=pd.concat([X_test, y_test], axis=1)
X_oot['ind']='oot'
oot=pd.concat([X_oot, y_oot], axis=1)
card = pd.concat([train, test,oot], axis=0)

In [55]:
#card.to_csv('card.csv', index=False)

In [ ]:
card=pd.read_csv('card.csv')

In [56]:
#### Data distribution summary

data_dist=card.assign(temp_good=np.where(card['good_bad']==1,card['wgt'],0),
            temp_bad=np.where(card['good_bad']==0,card['wgt'],0)
           ).groupby('ind',sort=False,as_index=False).agg(Raw=('ind','count'),Total=('wgt',sum),
                                  Good=('temp_good',sum),Bad=('temp_bad',sum))
data_dist['Total']=data_dist['Total'].astype('int')
data_dist['Good']=data_dist['Good'].astype('int')
data_dist['Bad']=data_dist['Bad'].astype('int')
data_dist['Bad_Rate']=round((data_dist.Bad/data_dist.Total)*100,1)
data_dist
### target 1---> good  ,0 ----> bad

/var/folders/nb/r3vglnfj14v41b4lyl2zgxbh0000gn/T/ipykernel_8827/4036885351.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).groupby('ind',sort=False,as_index=False).agg(Raw=('ind','count'),Total=('wgt',sum),


,ind,Raw,Total,Good,Bad,Bad_Rate
0,train,184525,184525,159303,25222,13.7
1,test,46132,46132,39827,6305,13.7
2,oot,235628,235628,216187,19441,8.3


#### Categorical variable (i.e Nominal Variable) WOE calculation

In [ ]:
from optbinning import OptimalBinning

In [57]:
predictors=['grade','sub_grade','emp_length_int','home_ownership','purpose','addr_state','dti','delinq_2yrs','inq_last_6mths',
            'mths_since_last_delinq','mths_since_last_record','pub_rec','revol_util','live_accts_ratio',
            'mths_since_earliest_cr_line']
cols=['wgt','good_bad']

# Optimal Binning Process

In [58]:
from sklearn.linear_model import LogisticRegression
from optbinning import BinningProcess

#####  Make sure all the columns have correct datatype assigned

In [59]:
card['emp_length_int'] = pd.to_numeric(card['emp_length_int'])
card['emp_length_int'] = card['emp_length_int'].astype(float)

In [60]:
card[predictors].info()

<class 'pandas.core.frame.DataFrame'>
Index: 466285 entries, 184676 to 466284
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   grade                        466285 non-null  object 
 1   sub_grade                    466285 non-null  object 
 2   emp_length_int               466285 non-null  float64
 3   home_ownership               466285 non-null  object 
 4   purpose                      466285 non-null  object 
 5   addr_state                   466285 non-null  object 
 6   dti                          466285 non-null  float64
 7   delinq_2yrs                  466285 non-null  float64
 8   inq_last_6mths               466285 non-null  float64
 9   mths_since_last_delinq       466285 non-null  float64
 10  mths_since_last_record       466285 non-null  float64
 11  pub_rec                      466285 non-null  float64
 12  revol_util                   466285 non-null  float64
 13 

In [68]:
categorical_columns = card[predictors].select_dtypes(include=['object']).columns.tolist()
numerical_columns = card[predictors].select_dtypes(include=['number']).columns.tolist()

In [69]:
special_codes = [-999]
#special_codes = {'special_1': -9, "special_2": -8, "special_3": -7}

In [70]:
#Dictionary with optimal binning fitting options for specific variables-we can update it as per our customization
# monotonicity constraints with respect to bad rate
#split_digits-- If split_digits is set to 0, the split points are integers
binning_fit_params = {
    "dti": {"monotonic_trend": "ascending","split_digits":2 ,
            "user_splits": [ 8.89, 10.91, 14.68, 16.03,18.23, 20.8 , 22.11, 28.37],
           # "user_splits_fixed" :[True,True] 
           }
    }


In [70]:
### after finalizing binings and transformation strategy - 
### by default special values has given 0 woe and 0 event rate Hence update it as per our requirement.
### with carefully for all the final variable specify transform woe strategy .
### we can neutralize special values by assigning woe==0
### to keep observed woe for special values assign metric_special':'empirical and 
## to merge with any other variable then assign woe of that particular bin
binning_transform_params ={
   'revol_util':{'metric_special':'empirical'},
    'dti':{'metric_special': -0.306345},
    'inq_last_6mths':{'metric_special':'empirical'}
}

In [71]:
#List of variables to be fixed. The binning process will retain these variables if the selection criteria is not satisfied.
fixed_variables=['grade',
                 'mths_since_earliest_cr_line',
                'live_accts_ratio',
                'emp_length_int',
                 'mths_since_last_delinq',
                 'delinq_2yrs'
                ]

In [72]:
selection_criteria = {
    "iv": {"min": 0.01, "max": 0.5,"strategy": "highest", "top": 50},
    "quality_score": {"min": 0.01}
}

In [73]:
# list of variables which are not stable across Train/Test/OOT or variables that we manually need to drop
unstable_vars=[]

In [74]:
model_vars = [x for x in predictors if x not in unstable_vars]

In [76]:
binning_process = BinningProcess(model_vars, special_codes=special_codes,
                                 categorical_variables=categorical_columns,
                                 selection_criteria=selection_criteria,
                                 binning_fit_params=binning_fit_params,
                                 #binning_transform_params=binning_transform_params,
                                fixed_variables=fixed_variables)

In [77]:
binning_process.fit(X=card[card['ind']=='train'][model_vars], y=1-card[card['ind']=='train']['good_bad'].values,sample_weight=card[card['ind']=='train']['wgt'])

/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_

BinningProcess(binning_fit_params={'dti': {'monotonic_trend': 'ascending',
                                           'split_digits': 2,
                                           'user_splits': [8.89, 10.91, 14.68,
                                                           16.03, 18.23, 20.8,
                                                           22.11, 28.37]}},
               categorical_variables=['grade', 'sub_grade', 'home_ownership',
                                      'purpose', 'addr_state'],
               fixed_variables=['grade', 'mths_since_earliest_cr_line',
                                'live_accts_ratio', 'emp_length_int',
                                'mths_since_last_delin...
               selection_criteria={'iv': {'max': 0.5, 'min': 0.01,
                                          'strategy': 'highest', 'top': 50},
                                   'quality_score': {'min': 0.01}},
               special_codes=[-999],
               variable_names=['grade', 'sub_grade', 'emp_length_int',
                               'home_ownership', 'purpose', 'addr_state', 'dti',
                               'delinq_2yrs', 'inq_last_6mths',
                               'mths_since_last_delinq',
                               'mths_since_last_record', 'pub_rec',
                               'revol_util', 'live_accts_ratio',
                               'mths_since_earliest_cr_line'])

In [78]:
# We can save all the variables passed to binning process and their IVs for manual screening
iv_tab=binning_process.summary().sort_values(by='iv',ascending=False)
#iv_tab.to_excel('iv_tab.xlsx', index=False)
iv_tab

,name,dtype,status,selected,n_bins,iv,js,gini,quality_score
1,sub_grade,categorical,OPTIMAL,True,13,0.296286,0.03589,0.299387,0.440846
0,grade,categorical,OPTIMAL,True,5,0.278102,0.03379,0.28495,0.811879
8,inq_last_6mths,numerical,OPTIMAL,True,4,0.04244,0.005284,0.109358,0.136384
12,revol_util,numerical,OPTIMAL,True,11,0.036151,0.004501,0.105994,0.045964
4,purpose,categorical,OPTIMAL,True,4,0.034385,0.004278,0.091372,0.0972
6,dti,numerical,OPTIMAL,True,9,0.030626,0.003819,0.099676,0.079235
3,home_ownership,categorical,OPTIMAL,True,3,0.01717,0.002145,0.0678,0.050685
14,mths_since_earliest_cr_line,numerical,OPTIMAL,True,10,0.016977,0.002119,0.072662,0.007304
5,addr_state,categorical,OPTIMAL,False,10,0.012468,0.001556,0.061741,0.00277
13,live_accts_ratio,numerical,OPTIMAL,True,9,0.011192,0.001398,0.059706,0.002779


##### To check whether transformation is perfectly working fine or not

In [ ]:
#### To check whether transformation is perfectly working fine or not

df=pd.DataFrame({'grade': ['A','B'],'sub_grade':['F3','B3'],
             'purpose':['credit_card','credit_card'], 'dti':[7.24,-999],'inq_last_6mths':[0,-999]
              ,'revol_util':[93,-999],
                'home_ownership':[-999,np.nan]})
df




In [205]:
special_codes = [-999]
binning_fit_params_oot = {}
for var in binning_process.get_support(names=True):
    binning_fit_params_oot[var] ={"user_splits": binning_process._binned_variables[var].splits}



In [211]:
oot_vars=binning_process.get_support(names=True)

In [213]:
binning_process_oot = BinningProcess(oot_vars, special_codes=special_codes,
                                 binning_fit_params=binning_fit_params_oot
                                 )

In [214]:
binning_process_oot.fit(X=card[card['ind']=='oot'][oot_vars], y=1-card[card['ind']=='oot']['good_bad'].values,sample_weight=card[card['ind']=='oot']['wgt'])

/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_

BinningProcess(binning_fit_params={np.str_('delinq_2yrs'): {'user_splits': array([0.5])},
                                   np.str_('dti'): {'user_splits': array([ 8.89, 10.91, 14.68, 16.03, 18.23, 20.8 , 22.11, 28.37])},
                                   np.str_('emp_length_int'): {'user_splits': array([5.5, 6.5, 7.5, 9.5])},
                                   np.str_('grade'): {'user_splits': [array(['A'], dtype=object),
                                                                      array(['B'], dtype=object),
                                                                      array(['C'], dtype=object)...
                                                                          array(['E1', 'E3', 'E2', 'E5'], dtype=object),
                                                                          array(['E4', 'F1', 'G4', 'F2', 'G5', 'F3', 'G2', 'F4', 'G1', 'F5', 'G3'],
      dtype=object)]}},
               special_codes=[-999],
               variable_names=array(['grade', 'sub_grade', 'emp_length_int', 'home_ownership',
       'purpose', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'revol_util', 'live_accts_ratio',
       'mths_since_earliest_cr_line'], dtype='<U27'))

In [78]:
#binning_process.transform(df,metric="woe")

In [79]:
optb = binning_process.get_binned_variable('sub_grade')
df = optb.binning_table.build()
df

/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"[A1, A2, A3, A4]",22233,0.120488,21179,1054,0.047407,1.157326,0.105501,0.012498
1,"[A5, B1]",18613,0.100870,17249,1364,0.073282,0.694241,0.037627,0.004611
2,[B2],12109,0.065623,11041,1068,0.088199,0.492736,0.013286,0.001644
3,[B3],14844,0.080444,13353,1491,0.100445,0.349203,0.008628,0.001073
4,[B4],13578,0.073584,12013,1565,0.115260,0.195012,0.002606,0.000325
5,[B5],9476,0.051353,8254,1222,0.128957,0.067118,0.000226,0.000028
6,[C1],10759,0.058306,9332,1427,0.132633,0.034784,0.000070,0.000009
7,[C2],10294,0.055786,8856,1438,0.139693,-0.025249,0.000036,0.000004
8,"[C3, C4, C5]",26014,0.140978,21866,4148,0.159453,-0.180785,0.004917,0.000614
9,"[D1, D2, D3]",17858,0.096778,14391,3467,0.194143,-0.419778,0.019781,0.002455


In [ ]:
binning_process.transform(X=card[(card['ind']=='train') & (card['delinq_2yrs']==-999)][predictors].head(), metric="woe")

In [217]:
card[(card['ind']=='oot') & (card['delinq_2yrs']==-999)][predictors].head()

,grade,sub_grade,emp_length_int,home_ownership,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_util,live_accts_ratio,mths_since_earliest_cr_line


In [83]:
!pwd

/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython


### Bivariate table  and Plot

In [215]:
# There is scope of improvement like currently plot is getting cropped so using seaborn we can customized
def bivariate_plot(filename='bivariate',binning_process=binning_process):
    with pd.ExcelWriter(f'{filename}.xlsx', engine='openpyxl', mode='w') as writer:
        for var in binning_process.get_support(names=True):
            optb = binning_process.get_binned_variable(var)
            
            # Get DataFrame from binning table
            df = optb.binning_table.build()
            df = df.drop(columns='JS')
            df = df.reset_index()
            
            # Write DataFrame to Excel
            df.to_excel(writer, sheet_name=var, index=False)
            # Plotting
            plot_image_path = f'variable_{var}_plot.png'
            optb.binning_table.plot(metric="event_rate", show_bin_labels=True,savefig=plot_image_path)
            #plt.close()
    
            # Insert plot image into the Excel sheet
            img = Image(plot_image_path)
            img.anchor = f'K1'  # Adjust the anchor cell as needed
            writer.sheets[var].add_image(img)
    
    for var in binning_process.get_support(names=True):
        plot_image_path = f'variable_{var}_plot.png'
        os.remove(plot_image_path)
        


In [108]:
bivariate_plot()

/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_

In [216]:
bivariate_plot('bivariate_oot',binning_process_oot)

/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_

In [ ]:
card.good_bad

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,emp_length_int,earliest_cr_line_date,issue_date,mths_since_earliest_cr_line,term_int,live_accts_ratio,wgt,ind,good_bad
184676,184676,2079703,2432175,35000,35000,35000.0,60 months,23.63,999.38,F,F3,GRJJC,3 years,RENT,98000.0,Verified,01-11-2012,Late (31-120 days),n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 11/16/12 > Need to consoli...,debt_consolidation,debt consolidation,760xx,TX,23.72,0.0,01-10-1990,0.0,45.0,-999.0,18.0,0.0,41864,73.6,47.0,f,20781.26,20781.26,32959.380000,32959.38,14218.75,18740.63,0.0,0.0,0.0,01-09-2015,999.38,Feb-16,01-01-2016,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,62.0,204072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56900.0,NaN,NaN,NaN,3.0,1990-10-01,2012-11-01,265.0,60,0.38,1,train,0
56860,56860,8634706,10406845,14075,14075,14075.0,60 months,21.00,380.78,E,E1,Certified Sorter,9 years,RENT,37920.0,Verified,01-11-2013,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Consolidation Loan,274xx,NC,7.85,0.0,01-10-2003,0.0,65.0,-999.0,13.0,0.0,8073,76.2,25.0,f,9903.05,9903.05,9518.840000,9518.84,4171.95,5346.89,0.0,0.0,0.0,01-12-2015,380.78,Jan-16,01-01-2016,0.0,65.0,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,35997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10600.0,NaN,NaN,NaN,9.0,2003-10-01,2013-11-01,121.0,60,0.52,1,train,1
212193,212193,1362521,1606495,9125,9125,9125.0,36 months,12.12,303.61,B,B3,KCI USA Inc.,7 years,RENT,20000.0,Verified,01-06-2012,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 06/13/12 > I am trying wan...,credit_card,Card Consolidation,597xx,MT,26.82,0.0,01-10-2001,0.0,-999.0,-999.0,7.0,0.0,13145,76.9,16.0,f,0.00,0.00,9307.360000,9307.36,9125.00,182.36,0.0,0.0,0.0,01-08-2012,9005.06,NaN,01-08-2012,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9125.0,NaN,NaN,NaN,7.0,2001-10-01,2012-06-01,128.0,36,0.44,1,train,1
7098,7098,887533,1103706,7200,7200,7200.0,36 months,6.62,221.07,A,A2,Clear Channel Communications,7 years,MORTGAGE,78000.0,Not Verified,01-09-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 09/18/11 > Some additional...,debt_consolidation,Personal Loan,181xx,PA,11.02,0.0,01-11-1991,1.0,-999.0,-999.0,16.0,0.0,7501,26.7,38.0,f,0.00,0.00,7938.148363,7938.15,7200.00,738.15,0.0,0.0,0.0,01-05-2014,1324.81,NaN,01-01-2016,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7200.0,NaN,NaN,NaN,7.0,1991-11-01,2011-09-01,238.0,36,0.42,1,train,1
199243,199243,1519024,1781883,10000,10000,10000.0,36 months,7.62,311.62,A,A3,NaN,NaN,MORTGAGE,107500.0,Verified,01-09-2012,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 09/04/12 > I want to pay o...,debt_consolidation,My no-debt plan,300xx,GA,12.26,0.0,01-03-1991,3.0,-999.0,-999.0,9.0,0.0,39346,32.0,20.0,f,0.00,0.00,10774.265960,10774.27,10000.00,774.27,0.0,0.0,0.0,01-07-2014,457.77,NaN,01-01-2016,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,211277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

# Variable stability check

In [114]:
from optbinning.binning.metrics import *

In [115]:
## Function to get vsi for x_oot data basis x_train data
def vsi_check(X_oot, X_train,binning_process,style='summary',psi_min_bin_size=0.01):
    variables = binning_process.get_support(names=True)
    
    #sc_table = self.scorecard.table()

    l_df_psi = []

    for name in variables:
        
        optb = binning_process.get_binned_variable(name)
        sc_table = optb.binning_table.build()
        ta = optb.transform(X_oot[name], metric="bins")
        te = optb.transform(X_train[name], metric="bins")


        n_bins = len(np.unique(te))

        n_records_a = np.empty(n_bins).astype(np.int64)
        n_records_e = np.empty(n_bins).astype(np.int64)

        for index,i in enumerate(sc_table["Bin"].values[:n_bins]):
            n_records_a[index] = np.count_nonzero(ta == str(i))
            n_records_e[index] = np.count_nonzero(te == str(i))

        t_n_records_a = n_records_a.sum()
        t_n_records_e = n_records_e.sum()
        p_records_a = n_records_a / t_n_records_a
        p_records_e = n_records_e / t_n_records_e

        psi = jeffrey(p_records_a, p_records_e, return_sum=False)
        bins = sc_table["Bin"].values[:n_bins]

        df_psi = pd.DataFrame({
            "Variable": [name] * n_bins,
            "Bin": bins,
            "Count OOT": n_records_a,
            "Count Train": n_records_e,
            "Count OOT (%)": p_records_a,
            "Count Train (%)": p_records_e,
            "PSI": psi
            })

        l_df_psi.append(df_psi)

    df_psi_variable = pd.concat(l_df_psi)
    df_psi_variable.reset_index()

    if style == "summary":
        return pd.DataFrame(
            df_psi_variable[df_psi_variable["Count Train (%)"]>=psi_min_bin_size].groupby(['Variable'])['PSI'].sum()
            ).reset_index()
    elif style == "detailed":
        return df_psi_variable

In [120]:
vsi_df_detail=vsi_check(card[card['ind']=='oot'][model_vars],
                        card[card['ind']=='train'][model_vars],
                        binning_process=binning_process,
                        style = "detailed")
vsi_df_summ=vsi_check(card[card['ind']=='oot'][model_vars],
                      card[card['ind']=='train'][model_vars],
                      binning_process=binning_process)


/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/sandip/Documents/PythonProjects/PythonApps/CreditRiskModellingPython/.venv/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_

In [121]:
vsi_df_detail

,Variable,Bin,Count OOT,Count Train,Count OOT (%),Count Train (%),PSI
0,grade,[A],36108,31072,0.153242,0.168389,1.427839e-03
1,grade,[B],61935,59781,0.262851,0.323972,1.277880e-02
2,grade,[C],66565,47067,0.282500,0.255071,2.801554e-03
3,grade,[D],42991,27185,0.182453,0.147324,7.512474e-03
4,grade,"[E, F, G]",28029,19420,0.118954,0.105243,1.679176e-03
0,sub_grade,"[A1, A2, A3, A4]",25347,22233,0.107572,0.120488,1.464460e-03
1,sub_grade,"[A5, B1]",21398,18613,0.090813,0.100870,1.056325e-03
2,sub_grade,[B2],11439,12109,0.048547,0.065623,5.146436e-03
3,sub_grade,[B3],13075,14844,0.055490,0.080444,9.267133e-03
4,sub_grade,[B4],13475,13578,0.057188,0.073584,4.133151e-03


In [122]:
# to save vsi in excel
with pd.ExcelWriter('vsi.xlsx', engine='openpyxl', mode='w') as writer:
    vsi_df_detail.to_excel(writer, sheet_name='detail', index=False)
    vsi_df_summ.to_excel(writer, sheet_name='summary', index=False)

In [ ]:
vsi_df_summ

In [ ]:
## vsi drop list
vsi_cutoff=0.01
vsidropp=list(vsi_df_summ[vsi_df_summ['PSI']>vsi_cutoff]['Variable'])
vsidropp 

### To check correleation between indpendent variables and dependent variables
###### Keep data type correct
###### We can remove highly associated or correleated independent variables 

- https://www.turintech.ai/four-methods-to-statistically-measure-your-data-correlation/#:~:text=Correlation%20Ratio&text=It%20can%20take%20values%20between%200%20and%201.,differences%20within%20the%20categorical%20data.
- For a measure of association between two categorical features. Introducing: Cramér’s V. It is based on a nominal variation of Pearson’s Chi-Square Test. Similarly to correlation, the output is in the range of [0,1], where 0 means no association and 1 is full association. (Unlike correlation, there are no negative values, as there’s no such thing as a negative association. Either there is, or there isn’t)
- For a measure of association between categorical and numerical variables ,we have correleation ratio.It is a measure of the correlation between a categorical column and a numeric column. It measures the variance of the mean of the numeric column across different categories of the categorical column. Unlike the Pearson and Spearman Correlations, the Correlation Ratio is unable to measure the “direction” of the correlation, and can only measure its magnitude. It can take values between 0 and 1. A value of 1 indicates that the variance in the numeric data is purely due to the difference within the categorical data. A value of 0 indicates that the variance in the numeric data is completely unaffected by any differences within the categorical data

In [85]:
from dython.nominal import associations,cluster_correlations

In [86]:
df=card[card['ind']=='train'][['good_bad'] + predictors]

In [ ]:
df .info()

In [ ]:
corr_matrix=associations(df, figsize=(15, 15))

In [ ]:
corr_matrix['corr']

In [57]:
correlations, _ = cluster_correlations(corr_matrix['corr'])

In [ ]:
correlations

## EnSelection
##### Enselection gives correleated variables hence manually need to remove highly correleated independent variables
##### Select highly correleated independent variable with dependent variables
###


- ##### Select the optimal hyperparameter C from validation curve which is not underfitting and overfitting
- ##### l1-ratio=0.99 is fixed for more penalty of l1

In [ ]:
# for enselection keeps l1_ratio close to 1 i.e 0.95,0.99, (i.e more Lasso penalty )
# l1_ratio is combination of L1 and L2 penalty
# alpha is regularization parameter

In [90]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import validation_curve, train_test_split
import pandas as pd
import numpy as np

In [91]:
X_woe_train = binning_process.transform(X=card[card['ind']=='train'][predictors], metric="woe")
y_woe_train = card[card['ind']=='train']['good_bad'].values

In [156]:
X_woe_test = binning_process.transform(X=card[card['ind']=='test'][predictors], metric="woe")
y_woe_test = card[card['ind']=='test']['good_bad'].values

In [ ]:
X_woe_test.head()

In [ ]:
# Define the range of values for the hyperparameter (C)
param_range = [0.01,0.1,1,10,100,1000]

fit_params_ = {'sample_weight':card[card['ind']=='train']['wgt'] # Example of passing sample weights
             }

# Create a LogisticRegression model with elastic net penalty
model = LogisticRegression(penalty='elasticnet',l1_ratio=0.99, solver='saga', random_state=42)

# Use validation_curve to generate training and validation scores
train_scores, test_scores = validation_curve(
    model, X_woe_train, y_woe_train, param_name='C', param_range=param_range,
    cv=5, scoring='roc_auc', n_jobs=-1,fit_params=fit_params_
)

# Plot the validation curve
plt.figure(figsize=(10, 6))
plt.plot(param_range, np.mean(train_scores, axis=1), label='Training Score', marker='o')
plt.plot(param_range, np.mean(test_scores, axis=1), label='Validation Score', marker='o')

plt.xscale('log')  # Use a log scale for C values
plt.xlabel('C (Inverse of Regularization Strength)')
plt.ylabel('ROC_AUC Score')
plt.title('Validation Curve for Logistic Regression with Elastic Net Penalty')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Define the Choosen hyperparameters basis validation curve
C = 0.1 # Regularization strength, you can adjust this as well
#l1_ratio = 0.99  # You can adjust this value based on your preference
sample_weight=card[card['ind']=='train']['wgt'] # Example of passing sample weights
             

# Create and fit the logistic regression model with elastic net penalty
en_model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.99, C=C, random_state=42)
en_model.fit(X_woe_train, y_woe_train,sample_weight=sample_weight)



In [ ]:
# Extract selected variables and their coefficients basis coefficient cut of >0.05
selected_variables = []
coefficients = []

# Iterate through the features and their coefficients
for feature, coef in zip(X_woe_train.columns, en_model.coef_[0]):
    if coef > abs(0.05):
        selected_variables.append(feature)
        coefficients.append(coef)

# Create a DataFrame to display the results
results_df = pd.DataFrame({
    'Variable': selected_variables,
    'Coefficient': coefficients
})


enVars =  results_df.reindex(results_df['Coefficient'].abs().sort_values(ascending=False).index)
enVars = enVars.reset_index(drop=True)
enVars


# Removing Correleated independent Variables using VIF and Corr 

In [ ]:
# to find correlection between independent variabls( This will be helful for if dummy variable
# List of columns for which logistic regression will be fitted
columns_to_fit = enVars['Variable'].values

# Dictionary to store linear predictors
linear_predictors = {}

# Iterate through columns and fit logistic regression without intercept
for col in columns_to_fit:
    X = X_woe_train[col].values.reshape(-1, 1)  # Reshape to 2D array
    y = y_woe_train
    
    # Create and fit the logistic regression model without intercept
    model = LogisticRegression(fit_intercept=False, random_state=42)
    model.fit(X, y)
    
    # Store linear predictors in the dictionary
    linear_predictors[col] = model.decision_function(X)

# Convert the dictionary to a DataFrame
linear_predictors_df = pd.DataFrame(linear_predictors)

In [ ]:
df_corr=linear_predictors_df.corr()
df_corr

- ##### remove highly correleated vars and having low iV
- ##### corr cutoff>0.8
- ##### created final_envars variable list to pass building logistic regression


In [112]:
# for WOE transformed dataframe we can use below direct corr matrix
#X_woe_train[enVars['Variable'].values].corr()

In [98]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

In [102]:
VIF_DF=X_woe_train[enVars['Variable'].values]

In [ ]:
vif_data = pd.DataFrame() 
vif_data["feature"] = VIF_DF.columns 

# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(VIF_DF.values, i) 
                          for i in range(len(VIF_DF.columns))] 


vif_data =  vif_data.reindex(vif_data['VIF'].abs().sort_values(ascending=False).index)
vif_data = vif_data.reset_index(drop=True)
vif_data

# Final Variable list for Logistic regression

In [115]:
#while removing highly correleated variables drop variable with low iv
#iv_data=iv_tab[['name','iv']].set_index('name')

In [716]:
###
corr_var_remove_list=['grade']
final_list = [item for item in enVars.Variable if item not in corr_var_remove_list]

In [ ]:
final_list

# Final Logistic Regression Model

In [ ]:
#pass final_var_list as per expectation
# Define the Choosen hyperparameters basis validation curve
C = 0.1 # Regularization strength, you can adjust this as well
#l1_ratio = 0.99  # You can adjust this value based on your preference
sample_weight=card[card['ind']=='train']['wgt'] # Example of passing sample weights
             
# Create and fit the logistic regression model with elastic net penalty
log_model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.99, C=C, random_state=42)
log_model.fit(X_woe_train[final_var_list], y_woe_train,sample_weight=sample_weight)



In [ ]:
summary_table = pd.DataFrame(columns = ['Feature name'], data = final_var_list)
# Creates a dataframe with a column titled 'Feature name' and row values contained in the 'feature_name' variable.
summary_table['Coefficients'] = np.transpose(log_model.coef_)
# Creates a new column in the dataframe, called 'Coefficients',
# with row values the transposed coefficients from the 'LogisticRegression' object.
summary_table.index = summary_table.index + 1
# Increases the index of every row of the dataframe with 1.
summary_table.loc[0] = ['Intercept', log_model.intercept_[0]]
# Assigns values of the row with index 0 of the dataframe.
summary_table = summary_table.sort_index()
# Sorts the dataframe by index.
summary_table

# Logistic regression with P-values

In [197]:
# P values for sklearn logistic regression.

# Class to display p-values for logistic regression in sklearn.

from sklearn import linear_model
import scipy.stats as stat

class LogisticRegression_with_p_values:
    
    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y,sample_weight=None):
        self.model.fit(X,y,sample_weight)
        
        #### Get p-values for the fitted model ####
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X / denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores] ### two tailed test for p-values
        
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values

In [198]:
C = 0.1 # Regularization strength, you can adjust this as well
#l1_ratio = 0.99  # You can adjust this value based on your preference
sample_weight=card[card['ind']=='train']['wgt'] # Example of passing sample weights
             
# Create and fit the logistic regression model with elastic net penalty
log_model_pval = LogisticRegression_with_p_values(penalty='elasticnet', solver='saga', l1_ratio=0.99, C=C, random_state=42)
log_model_pval.fit(X_woe_train[final_var_list], y_woe_train,sample_weight=sample_weight)


In [ ]:
summary_table = pd.DataFrame(columns = ['Feature name'], data = final_var_list)
# Creates a dataframe with a column titled 'Feature name' and row values contained in the 'feature_name' variable.
summary_table['Coefficients'] = np.transpose(log_model.coef_)
# Creates a new column in the dataframe, called 'Coefficients',
# with row values the transposed coefficients from the 'LogisticRegression' object.
summary_table.index = summary_table.index + 1
# Increases the index of every row of the dataframe with 1.
summary_table.loc[0] = ['Intercept', log_model.intercept_[0]]
# Assigns values of the row with index 0 of the dataframe.
summary_table = summary_table.sort_index()
# We add the 'p_values' here, just as we did before.
p_values = log_model_pval.p_values
p_values = np.append(np.nan,np.array(p_values))
summary_table['p_values'] = p_values
summary_table['Significane']=summary_table.p_values.map(lambda x: 'No' if x>0.05 else 'Yes')
summary_table

In [ ]:
#probability of 1 (i.e good)
log_model_pval.model.predict_proba(X_woe_test[final_var_list])[:,1]

In [ ]:
X_woe_test[final_var_list].head()

In [ ]:
1.842504+(0.875628*0.035291)+(0.688286*0.100974)+(0.564239*-0.048576)+(0.517379*-0.141723)+\
(0.473847*0.192496)+(0.475539*-0.007919)+(0.329518*0.066615)

In [ ]:
1/(1+np.exp(-1.951570500972))

In [ ]:
log_model_pval.model.predict_proba

# KS GINI ROC_AUC on Train/Test/OOT

In [203]:
card_X_woe = binning_process.transform(X=card[predictors], metric="woe")
card_Y = card['good_bad'].values

In [204]:
#probability of 1 (i.e good)
card_pred_prodb=log_model_pval.model.predict_proba(card_X_woe[final_var_list])[:,1]

In [205]:
card_X_woe['logi_pred_prob_good']=card_pred_prodb
card_X_woe['good_bad']=card_Y
card_X_woe['ind']=card['ind'].values
card_X_woe['wgt']=card['wgt'].values

In [ ]:
card_X_woe.head()

In [576]:
from sklearn.metrics import roc_curve, roc_auc_score

In [705]:
#Final Gain table code

def gainTable(data,score,ground_truth,weights=None,response_name=['Good','Bad'],numOfIntervals=10,breaks=None):
    """
    This function gives K-S statistics Tables 
    KS Statistics is the difference between the cumulative Success and Non-Success Rate.Which gives optimal threshold for the group separation
    inuputs:
    data:dataframe 
    score:string,coulmn name which contains predicted probability or score from the model
    ground_truth:string,column name which contains actual labels in integer form
    weights:array,column name contains weights for the sample/row,if None then default weight=1 is passed
    response_name:list,list of  of your response & non-response label.Pass first element name for 1 and then 0 e.g.['Good','Bad'] if 1 is Good
    numOfIntervals:int,for number of intervals required
    breaks:array of breaks ,default None
    returns:
    GainsTable:dataframe
    Breaks: array
    """
    #Sort the data in ascending order of predicted probabilities.
    data= data.sort_values(by=score, ascending=True)
    if weights is None:
        data['wgt']=1
        weights='wgt'
    if breaks is None:
        result=data.groupby(score).agg( #Group by Deciles of Predicted Probabilties
            [        
                'sum'
            ]
            )[weights].sort_index(ascending=True)
        result = result.rename(columns={'sum': 'TotalWeight'})
        result = result.reset_index()
        result['cum_weights']=result['TotalWeight'].cumsum()
        
        # Calculate the weighted ECDF
        x_ = result[score].values
        y_ = result['cum_weights'] / result['TotalWeight'].sum()
        # Use interpolation to find the values corresponding to the quantiles
        quantiles = np.linspace(0, 1, num=numOfIntervals+1)
        breaks = np.unique(np.interp(quantiles, y_, x_))
        breaks[0]  = -np.inf
        breaks[-1] = np.inf
        if data[score].max()>1:
            breaks = np.round(breaks)
        

        
    data['Score_Range'] = pd.cut(data[score], bins=breaks,include_lowest=True,duplicates='drop')
        
    #print(breaks)
    #Create success and failure response column
    wm = lambda x: np.average(x, weights=data.loc[x.index, weights])
    

    KS_data = data.assign(temp_good=np.where(data[ground_truth]==1,data[weights],0),
                          temp_bad=np.where(data[ground_truth]==0,data[weights],0)
                         ).groupby('Score_Range').agg(Totals=(weights,sum),
                                  Good=('temp_good',sum),
                                  Bad=('temp_bad',sum),
                                  AvgScore=(score, wm)).sort_index(ascending=True)
    
    KS_data.columns = ['Totals',response_name[0],response_name[1],'AvgScore']
    KS_data[response_name[1]+'_Rate'] = (KS_data[response_name[1]] / KS_data['Totals']).apply(lambda x:round(100*x,2))
    KS_data[response_name[0]+'_Rate'] = (KS_data[response_name[0]] / KS_data['Totals']).apply(lambda x:round(100*x,2))
    
    KS_data['Cum'+response_name[1]] = np.round((KS_data[response_name[1]] / KS_data[response_name[1]].sum()).cumsum(),4)*100
    KS_data['Cum'+response_name[0]] = np.round((KS_data[response_name[0]] / KS_data[response_name[0]].sum()).cumsum(),4)*100
    KS_data['KS'] = np.round(abs(KS_data['Cum'+response_name[1]]-KS_data['Cum'+response_name[0]]), 4)
    KS_data=KS_data.sort_index(ascending=False)
    KS_data['Approval%' ] = np.round((KS_data['Totals'] / KS_data['Totals'].sum()).cumsum(), 4) * 100
    KS_data['Approval'+ response_name[1]+'Rate' ] = np.round((KS_data[response_name[1]].cumsum() / KS_data['Totals'].cumsum()), 4) * 100
    KS_data=KS_data.sort_index(ascending=True)   
    #Calculate Lift = Ratio of Bads to the number of data points in the decile
    KS_data['Lift'] = (KS_data['Cum'+response_name[1]]/((KS_data['Totals'].cumsum()/KS_data['Totals'].sum())*100)).apply(lambda x:round(x,2))  

    KS_data=KS_data[['Totals',response_name[1],response_name[1]+'_Rate','Cum'+response_name[1],
                              response_name[0],response_name[0]+'_Rate','Cum'+response_name[0],
                              'KS','AvgScore','Approval%','Approval'+ response_name[1]+'Rate','Lift']]
    KS_data.reset_index(inplace=True)
    sum_row = {'Score_Range': 'Total',
           'Totals': KS_data['Totals'].sum(),
           response_name[1] : KS_data[response_name[1]].sum(),
           response_name[1]+'_Rate': np.round((KS_data[response_name[1]].sum()/KS_data['Totals'].sum())*100,2),
           response_name[0] : KS_data[response_name[0]].sum(),
           response_name[0]+'_Rate': np.round((KS_data[response_name[0]].sum()/KS_data['Totals'].sum())*100,2),
               'KS':KS_data['KS'].max(),
            'AvgScore':np.average(data[score], weights=data[weights])
                }
    KS_data.loc[len(KS_data)] = sum_row
    KS=KS_data['KS'].max()
    return {'GainTable':KS_data,'breaks':breaks,'KS':KS}



In [ ]:
## performance on Train/Test/OOT
perfs=[]
for seg in ['train','test','oot']:
    if seg=='train':
        GT=gainTable(data=card_X_woe[card_X_woe['ind']==seg],
        score='logi_pred_prob_good',
        weights='wgt',
        ground_truth='good_bad',
        response_name=['Good','Bad'],
        numOfIntervals=10,
        breaks=None)
        KS=GT['KS']
    else:
        GT_Train=gainTable(data=card_X_woe[card_X_woe['ind']=='train'],
        score='logi_pred_prob_good',
        weights='wgt',
        ground_truth='good_bad',
        response_name=['Good','Bad'],
        numOfIntervals=10,
        breaks=None)

        train_breaks=GT_Train['breaks']
        GT=gainTable(data=card_X_woe[card_X_woe['ind']==seg],
        score='logi_pred_prob_good',
        weights='wgt',
        ground_truth='good_bad',
        response_name=['Good','Bad'],
        numOfIntervals=10,
        breaks=train_breaks)
        KS=GT['KS']
        

    AUC_ROC = roc_auc_score(y_true=card_X_woe[card_X_woe['ind']==seg]['good_bad'], 
                        y_score=card_X_woe[card_X_woe['ind']==seg]['logi_pred_prob_good'],
                        sample_weight=card_X_woe[card_X_woe['ind']==seg]['wgt'])
    
    GINI=2*AUC_ROC-1
    AUC_ROC=np.round(AUC_ROC*100,2)
    GINI=np.round(GINI*100,2)
    perfs.append({'Seg':seg,'KS':KS,'AUC_ROC':AUC_ROC,'GINI':GINI        
    })
    
pd.DataFrame(perfs)    

# Train Gain Table

In [654]:
Train_GT=gainTable(data=card_X_woe[card_X_woe['ind']=='train'],score='logi_pred_prob_good',weights='wgt',ground_truth='good_bad',
             numOfIntervals=10)
#print(GT['GainTable'])
#print(GT['breaks'])

In [ ]:
Train_GT['GainTable']

In [ ]:
Train_GT['breaks']

# Test Gain Table

In [657]:
Test_GT=gainTable(data=card_X_woe[card_X_woe['ind']=='test'],
                  score='logi_pred_prob_good',
                  weights='wgt',
                  ground_truth='good_bad',
                  response_name=['Good', 'Bad'],
             numOfIntervals=10,breaks=Train_GT['breaks'])

In [ ]:
Test_GT['GainTable']
#Test_GT['KS']
#Test_GT['breaks']

# OOT Gain Table

In [661]:
OOT_GT=gainTable(data=card_X_woe[card_X_woe['ind']=='oot'],
                  score='logi_pred_prob_good',
                  weights='wgt',
                  ground_truth='good_bad',
                  response_name=['Good', 'Bad'],
             numOfIntervals=10,breaks=Train_GT['breaks'])

In [ ]:
OOT_GT['GainTable']
#Test_GT['KS']
#Test_GT['breaks']

# 3 Digit Score Conversion

In [673]:
def eqLinear(OddsAtAnchor,Anchor=600,PDO=20):
    alpha=Anchor- PDO/np.log(2)* np.log(OddsAtAnchor)
    beta= PDO/np.log(2)
    return {'alpha':np.round(alpha,4),'beta':np.round(beta,4)}

In [ ]:
eqLinear(OddsAtAnchor=3,Anchor=600,PDO=50)

In [695]:
def ThreedigitSocre(prob,alpha,beta):
    logodds=np.log(prob/(1-prob))
    score=(logodds*beta)+alpha
    return np.round(score,0)

In [700]:
card_X_woe['Score'] = card_X_woe['logi_pred_prob_good'].apply(lambda x: ThreedigitSocre(x,alpha= 520.7519,beta=72.1348))


In [ ]:
card_X_woe.head(5)

In [ ]:
card_X_woe[final_list+['logi_pred_prob_good'	,'good_bad','ind','wgt','Score']].head()

In [ ]:

#1/(1+np.exp(-0.903314*0.802530))
1/(1+np.exp(-0.196551*0.688542))

In [ ]:
#subgrade
print(ThreedigitSocre(0.67369315891759,alpha= 520.7519,beta=72.1348))
#subgrade
print(ThreedigitSocre(0.5337818602980491,alpha= 520.7519,beta=72.1348))

In [ ]:
573+531+

In [ ]:
summary_table

In [706]:
Train_GT=gainTable(data=card_X_woe[card_X_woe['ind']=='train'],score='Score',weights='wgt',ground_truth='good_bad',
             numOfIntervals=10)
#print(GT['GainTable'])
#print(GT['breaks'])

In [ ]:
Train_GT['GainTable']

In [ ]:
#########End ############

In [637]:
#Final Gain table code

def gainTable(data,score,ground_truth,weights=None,response_name=['Good','Bad'],numOfIntervals=10,breaks=None):
    """
    This function gives K-S statistics Tables 
    KS Statistics is the difference between the cumulative Success and Non-Success Rate.Which gives optimal threshold for the group separation
    inuputs:
    data:dataframe 
    score:string,coulmn name which contains predicted probability or score from the model
    ground_truth:string,column name which contains actual labels in integer form
    weights:array,column name contains weights for the sample/row,if None then default weight=1 is passed
    response_name:list,list of  of your response & non-response label.Pass first element name for 1 and then 0 e.g.['Good','Bad'] if 1 is Good
    numOfIntervals:int,for number of intervals required
    breaks:array of breaks ,default None
    returns:
    GainsTable:dataframe
    Breaks: array
    """
    #Sort the data in ascending order of predicted probabilities.
    data= data.sort_values(by=score, ascending=True)
    if weights is None:
        data['wgt']=1
        weights='wgt'
    if breaks is None:
        result=data.groupby(score).agg( #Group by Deciles of Predicted Probabilties
            [        
                'sum'
            ]
            )[weights].sort_index(ascending=True)
        result = result.rename(columns={'sum': 'TotalWeight'})
        result = result.reset_index()
        result['cum_weights']=result['TotalWeight'].cumsum()
        
        # Calculate the weighted ECDF
        x_ = result[score].values
        y_ = result['cum_weights'] / result['TotalWeight'].sum()
        # Use interpolation to find the values corresponding to the quantiles
        quantiles = np.linspace(0, 1, num=numOfIntervals+1)
        breaks = np.unique(np.interp(quantiles, y_, x_))
        breaks[0]  = -np.inf
        breaks[-1] = np.inf
        
    data['Score_Range'] = pd.cut(data[score], bins=breaks,include_lowest=True,duplicates='drop')
        
    #print(breaks)
    #Create success and failure response column
    wm = lambda x: np.average(x, weights=data.loc[x.index, weights])
    

    KS_data = data.assign(temp_good=np.where(data[ground_truth]==1,data[weights],0),
                          temp_bad=np.where(data[ground_truth]==0,data[weights],0)
                         ).groupby('Score_Range').agg(Totals=(weights,sum),
                                  Good=('temp_good',sum),
                                  Bad=('temp_bad',sum),
                                  AvgScore=(score, wm)).sort_index(ascending=True)
    
    KS_data.columns = ['Totals',response_name[0],response_name[1],'AvgScore']
    KS_data[response_name[1]+'_Rate'] = (KS_data[response_name[1]] / KS_data['Totals']).apply(lambda x:round(100*x,2))
    KS_data[response_name[0]+'_Rate'] = (KS_data[response_name[0]] / KS_data['Totals']).apply(lambda x:round(100*x,2))
    
    KS_data['Cum'+response_name[1]] = np.round((KS_data[response_name[1]] / KS_data[response_name[1]].sum()).cumsum(),4)*100
    KS_data['Cum'+response_name[0]] = np.round((KS_data[response_name[0]] / KS_data[response_name[0]].sum()).cumsum(),4)*100
    KS_data['KS'] = np.round(abs(KS_data['Cum'+response_name[1]]-KS_data['Cum'+response_name[0]]), 4)
    KS_data=KS_data.sort_index(ascending=False)
    KS_data['Approval%' ] = np.round((KS_data['Totals'] / KS_data['Totals'].sum()).cumsum(), 4) * 100
    KS_data['Approval'+ response_name[1]+'Rate' ] = np.round((KS_data[response_name[1]].cumsum() / KS_data['Totals'].cumsum()), 4) * 100
    KS_data=KS_data.sort_index(ascending=True)   
    #Calculate Lift = Ratio of Bads to the number of data points in the decile
    KS_data['Lift'] = (KS_data['Cum'+response_name[1]]/((KS_data['Totals'].cumsum()/KS_data['Totals'].sum())*100)).apply(lambda x:round(x,2))  

    KS_data=KS_data[['Totals',response_name[1],response_name[1]+'_Rate','Cum'+response_name[1],
                              response_name[0],response_name[0]+'_Rate','Cum'+response_name[0],
                              'KS','AvgScore','Approval%','Approval'+ response_name[1]+'Rate','Lift']]
    KS_data.reset_index(inplace=True)
    sum_row = {'Score_Range': 'Total',
           'Totals': KS_data['Totals'].sum(),
           response_name[1] : KS_data[response_name[1]].sum(),
           response_name[1]+'_Rate': np.round((KS_data[response_name[1]].sum()/KS_data['Totals'].sum())*100,2),
           response_name[0] : KS_data[response_name[0]].sum(),
           response_name[0]+'_Rate': np.round((KS_data[response_name[0]].sum()/KS_data['Totals'].sum())*100,2),
               'KS':KS_data['KS'].max(),
            'AvgScore':np.average(data[score], weights=data[weights])
                }
    KS_data.loc[len(KS_data)] = sum_row
    KS=KS_data['KS'].max()
    return {'GainTable':KS_data,'breaks':breaks,'KS':KS}


In [ ]:
import pandas as pd
import numpy as np

def k_s_statistics_gain_lift(data, predicted_probability, ground_truth, weights=None, response_name='Risk'):
    """
    This function gives K-S statistics Tables 
    KS Statistics is the difference between the cumulative Success and Non-Success Rate. 
    Which gives optimal threshold for the group separation in inputs:
    data: dataframe 
    predicted_probability: string, column name which contains predicted probability from the model
    ground_truth: string, column name which contains actual labels in integer form
    weights: string, column name which contains weights for each data point (optional)
    response_name: string, name of your success label e.g. default, fraud, churn etc
    """
    # Sort the data in descending order of predicted probabilities.
    data = data.sort_values(by=predicted_probability, ascending=False)
    
    # Cut deciles based on the predicted probabilities
    data['decile_group'] = pd.qcut(data[predicted_probability], q=10, duplicates='drop')
    
    # Create success and failure response column
    KS_data = data.groupby('decile_group').agg(  # Group by Deciles of Predicted Probabilities
        {
            'count',  # The total number of customers(data points) in the decile
            'sum',  # The total number of bad customers(Risk=1)
            weights if weights is not None else 'count'  # Use weights column if provided, else count
        }
    )[ground_truth].sort_index(ascending=False)

    KS_data.columns = ['Total count', 'Number of ' + response_name, 'Total weights']
    
    # Calculate the weighted number of non-response cases
    KS_data['Number of ' + 'Non-' + response_name] = KS_data['Total weights'] - KS_data['Number of ' + response_name]
    
    # Calculate percentages and round to 2 decimal places
    KS_data[response_name + '_Rate' + '%'] = (
        (KS_data['Number of ' + response_name] / KS_data['Total weights']) * 100
    ).round(2)
    
    KS_data['Percent of ' + response_name + '%'] = (
        (KS_data['Number of ' + response_name] / KS_data['Number of ' + response_name].sum()) * 100
    ).round(2)
    
    KS_data['Percent of ' + 'Non-' + response_name + '%'] = (
        (KS_data['Number of ' + 'Non-' + response_name] / KS_data['Number of ' + 'Non-' + response_name].sum()) * 100
    ).round(2)
    
    # Calculate KS statistics for weighted samples
    KS_data['ks_stats'] = np.round(
        ((KS_data['Number of ' + response_name].cumsum() / KS_data['Number of ' + response_name].sum()) -
         (KS_data['Number of ' + 'Non-' + response_name].cumsum() / KS_data['Number of ' + 'Non-' + response_name].sum())), 4
    ) * 100
    
    KS_data['max_ks'] = KS_data['ks_stats'].apply(lambda x: '*****' if x == KS_data['ks_stats'].max() else '')
    
    # Calculate Gain = Cumulative Percent of Events/Total success events
    KS_data['Gain'] = (KS_data['Percent of ' + response_name + '%'].cumsum()).round(2)
    
    # Calculate Lift = Ratio of Bads to the number of data points in the decile
    KS_data['Lift'] = (
        (KS_data['Gain'] / np.array(range(10, 100 + 10, 10))) * 100
    ).apply(lambda x: round(x, 2))
    
    return KS_data


In [ ]:

gains_result = gains_table(y_true, y_score, weights, num_buckets=10)


In [997]:
from sklearn.linear_model import LogisticRegression
from optbinning import BinningProcess
from optbinning import Scorecard
from optbinning.scorecard import ScorecardMonitoring

In [998]:
estimator = LogisticRegression(solver="lbfgs")

In [999]:
scorecard = Scorecard(binning_process=binning_process,
                      estimator=estimator, scaling_method="min_max",
                      scaling_method_params={"min": 300, "max": 900})

In [ ]:
scorecard.fit(X=card[card['ind']=='train'][model_vars], y=1-card[card['ind']=='train']['good_bad'].values)

In [1109]:
vsi_df_summ=vsi_check(card[card['ind']=='oot'][model_vars],card[card['ind']=='train'][model_vars],binning_process=binning_process)

In [1110]:
vsi_df_detail=vsi_check(card[card['ind']=='oot'][model_vars],card[card['ind']=='train'][model_vars],binning_process=binning_process,
         style = "detailed")

In [1016]:
optb = scorecard.binning_process.get_binned_variable("inq_last_6mths")
tab=optb.binning_table.build()


In [1005]:
monitoring = ScorecardMonitoring(scorecard=scorecard, psi_method="cart",
                                 psi_n_bins=10,verbose=True)

In [ ]:
monitoring.fit(card[card['ind']=='oot'][model_vars], 1-card[card['ind']=='oot']['good_bad'].values, 
               card[card['ind']=='train'][model_vars], 1-card[card['ind']=='train']['good_bad'].values)

In [ ]:
monitoring.psi_variable_table(style="detailed")

In [ ]:
for var in binning_process.get_support(names=True):

In [ ]:
pd.concat([card[['ind']].head().reset_index(drop = True),binning_process.transform(X=card[predictors].head().reset_index(drop = True), metric="bins")],axis=1)

In [823]:
X_transform = binning_process.transform(X=card[card['ind']=='train'][predictors].head(), metric="woe",metric_special='empirical')

In [ ]:
X_transform

In [ ]:
df=pd.DataFrame({'grade': ['A','B'],'sub_grade':['F3','B3'],
             'purpose':['credit_card','credit_card'], 'dti':[7.24,-999],'inq_last_6mths':[0,-999]
              ,'revol_util':[93,-999],
                'home_ownership':[-999,np.nan]})
df

In [ ]:
binning_process.transform(df,metric="indices")

In [ ]:
x_transform_woe = optb.transform(x_new, metric="woe")
pd.DataFrame({'grade': x_new, "WoE": x_transform_woe})

In [ ]:
card[card['ind']=='train'][binning_process.get_support(names=True)].head()

In [ ]:
binning_transform_params ={
    'revol_util':{'metric_special':-0.343823}
}

In [ ]:
update_variables = ["revol_util"]

for variable in update_variables:
    optb = OptimalPWBinning(estimator=LGBMClassifier(),
                            name=variable, objective="l1")
    optb.fit(X_train[variable], y_train, lb=0.001, ub=0.999)
    binning_process.update_binned_variable(name=variable, optb=optb)

In [ ]:
binning_process.fit(X=card[card['ind']=='train'][predictors], y=1-card[card['ind']=='train']['good_bad'].values,sample_weight=card[card['ind']=='train']['wgt'])

In [493]:
def woe_categorical_(card,discrete_variable_name,perf='good_bad'):
    df=pd.concat([card[card['ind']=='train'][discrete_variable_name],card[card['ind']=='train'][perf]],axis=1)
    
    df=pd.concat([df.groupby(df.columns.values[0],as_index=False)[df.columns.values[1]].count(),
         df.assign(temp_bad=np.where(df.iloc[:,[1]]==0,1,0)).groupby(df.columns.values[0],as_index=False).agg(Bad=('temp_bad','sum')),
         df.assign(temp_bad=np.where(df.iloc[:,[1]]==0,1,0)).groupby(df.columns.values[0],as_index=False).agg(Bad_Rate=('temp_bad','mean'))],
         axis=1)
    df=df.iloc[:,[0,1,3,5]]
    df.columns=[df.columns.values[0],'Count',df.columns.values[2],df.columns.values[3]]
    df['Good']=df['Count']-df['Bad']
    df['Good_Rate']=1-df['Bad_Rate']
    df['Count%']=df['Count']/(df['Count'].sum())
    df=df.iloc[:,[0,1,6,2,3,4,5]]
    df['Prop_Bad']=df['Bad']/df['Bad'].sum()
    df['Prop_Good']=df['Good']/df['Good'].sum()
    df['WOE']=np.log((df['Prop_Good'])/(df['Prop_Bad']))
    df = df.sort_values(['WOE'])
    df = df.reset_index(drop = True)
    #df['Diff_Prop_Good'] = df['Prop_Good'].diff().abs()
    #df['Diff_WOE'] = df['WOE'].diff().abs()
    df['IV'] = (df['Prop_Good'] - df['Prop_Bad']) * df['WOE']
    sum_row = {discrete_variable_name: 'Total',
           'Count': df['Count'].sum(),
           'Count%': df['Count%'].sum(),
           'Bad': df['Bad'].sum(),
          'Bad_Rate': df['Bad'].sum()/df['Count'].sum(),
          'Good': df['Good'].sum(),
          'Good_Rate': df['Good'].sum()/df['Count'].sum(),
          'IV': df['IV'].sum()}
    df.loc[len(df)] = sum_row
    return df
    
           
    



In [489]:
df_tmp=woe_categorical_(card,discrete_variable_name='home_ownership',perf='good_bad')

In [ ]:
df_tmp

### Preprocessing categorical  Variables: Visualizing woe Results

In [403]:
import matplotlib.pyplot as plt
import seaborn as sns
# Imports the libraries we need.
sns.set()
# We set the default style of the graphs to the seaborn style. 

In [407]:
# Below we define a function that takes 2 arguments: a dataframe and a number.
# The number parameter has a default value of 0.
# This means that if we call the function and omit the number parameter, it will be executed with it having a value of 0.
# The function displays a graph.
def plot_by_woe(df_WoE, rotation_of_x_axis_labels = 0):
    x = np.array(df_WoE.iloc[:, 0].apply(str))
    # Turns the values of the column with index 0 to strings, makes an array from these strings, and passes it to variable x.
    y = df_WoE['WOE']
    # Selects a column with label 'WoE' and passes it to variable y.
    plt.figure(figsize=(18, 6))
    # Sets the graph size to width 18 x height 6.
    plt.plot(x, y, marker = 'o', linestyle = '--', color = 'k')
    # Plots the datapoints with coordiantes variable x on the x-axis and variable y on the y-axis.
    # Sets the marker for each datapoint to a circle, the style line between the points to dashed, and the color to black.
    plt.xlabel(df_WoE.columns[0])
    # Names the x-axis with the name of the column with index 0.
    plt.ylabel('Weight of Evidence')
    # Names the y-axis 'Weight of Evidence'.
    plt.title(str('Weight of Evidence by ' + df_WoE.columns[0]))
    # Names the grapth 'Weight of Evidence by ' the name of the column with index 0.
    plt.xticks(rotation = rotation_of_x_axis_labels)
    # Rotates the labels of the x-axis a predefined number of degrees.

In [ ]:
plot_by_woe(df_tmp)

#### Continuous or discrete ordered variable woe

In [596]:
# WoE function for ordered discrete and continuous variables
def woe_ordered_continuous(df, ordered_discrete_continuous_variabe_name,
                           perf='good_bad',NumericToCategory=False,nbin=20,quantile_discretization=False):
    df=pd.concat([card[card['ind']=='train'][ordered_discrete_continuous_variabe_name],card[card['ind']=='train'][perf]],axis=1)
    if NumericToCategory:
        if quantile_discretization:
            df[ordered_discrete_continuous_variabe_name]=pd.qcut(df[ordered_discrete_continuous_variabe_name], q=nbin)
        else:
            df[ordered_discrete_continuous_variabe_name]=pd.cut(df[ordered_discrete_continuous_variabe_name], bins=nbin)
    
    df=pd.concat([df.groupby(df.columns.values[0],as_index=False)[df.columns.values[1]].count(),
         df.assign(temp_bad=np.where(df.iloc[:,[1]]==0,1,0)).groupby(df.columns.values[0],as_index=False).agg(Bad=('temp_bad','sum')),
         df.assign(temp_bad=np.where(df.iloc[:,[1]]==0,1,0)).groupby(df.columns.values[0],as_index=False).agg(Bad_Rate=('temp_bad','mean'))],
         axis=1)
    df=df.iloc[:,[0,1,3,5]]
    df.columns=[df.columns.values[0],'Count',df.columns.values[2],df.columns.values[3]]
    df['Good']=df['Count']-df['Bad']
    df['Good_Rate']=1-df['Bad_Rate']
    df['Count%']=df['Count']/(df['Count'].sum())
    df=df.iloc[:,[0,1,6,2,3,4,5]]
    df['Prop_Bad']=df['Bad']/df['Bad'].sum()
    df['Prop_Good']=df['Good']/df['Good'].sum()
    df['WOE']=np.log((df['Prop_Good'])/(df['Prop_Bad']))
    #df = df.sort_values(['WOE'])
    df = df.reset_index(drop = True)
    #df['Diff_Prop_Good'] = df['Prop_Good'].diff().abs()
    #df['Diff_WOE'] = df['WOE'].diff().abs()
    df['IV'] = (df['Prop_Good'] - df['Prop_Bad']) * df['WOE']
    sum_row = {ordered_discrete_continuous_variabe_name: 'Total',
           'Count': df['Count'].sum(),
           'Count%': df['Count%'].sum(),
           'Bad': df['Bad'].sum(),
          'Bad_Rate': df['Bad'].sum()/df['Count'].sum(),
          'Good': df['Good'].sum(),
          'Good_Rate': df['Good'].sum()/df['Count'].sum(),
          'IV': df['IV'].sum()}
    df.loc[len(df)] = sum_row
    return df
# Here we define a function similar to the one above, ...
# ... with one slight difference: we order the results by the values of a different column.
# The function takes 3 arguments: a dataframe, a string, and a dataframe. The function returns a dataframe as a result.

In [ ]:
 # term
card['emp_length_int'].unique()
# There are only two unique values, 36 and 60.

In [ ]:
df_tmp = woe_ordered_continuous(card, ordered_discrete_continuous_variabe_name='emp_length_int', perf='good_bad')
df_tmp

In [ ]:
plot_by_woe(df_tmp)

In [ ]:
card['dti_cat'].unique()

In [575]:
card['dti_cat'] = pd.cut(card['dti'], 20)

In [ ]:
df_tmp = woe_ordered_continuous(card, ordered_discrete_continuous_variabe_name='dti',
                                perf='good_bad',NumericToCategory=True,
                                quantile_discretization=False)
df_tmp

In [ ]:
plot_by_woe(df_tmp,45)

### by optbinning

In [388]:
#card["wgt"]=card.good_bad.map(lambda x: 10 if x==0 else 1)
#np.where(df.iloc[:,[1]]==0,1,0)

In [ ]:
#original_array.reshape(-1, 1)
user_splits = (card[card['ind']=='train']['purpose'].unique()).reshape(-1,1)
print(user_splits)

user_splits_fixed=len(card[card['ind']=='train']['purpose'].unique()) * [True]
print(user_splits_fixed)

In [503]:
optb = OptimalBinning(name='addr_state', dtype="categorical", solver="mip",user_splits=user_splits,
                      user_splits_fixed=user_splits_fixed)

In [ ]:
optb.fit(card[card['ind']=='train']['purpose'].values, 1-card[card['ind']=='train']['good_bad'].values,sample_weight=card[card['ind']=='train']['wgt'])

In [ ]:
binning_table = optb.binning_table
binning_table.build()


In [608]:
optb = OptimalBinning(name='grade', dtype="categorical", solver="mip")

In [ ]:
optb.fit(card[card['ind']=='train']['grade'].values, 1-card[card['ind']=='train']['good_bad'].values,sample_weight=card[card['ind']=='train']['wgt'])

In [ ]:
binning_table = optb.binning_table
binning_table.build()


In [ ]:
binning_table.plot(metric="woe",show_bin_labels=True)

In [617]:
x_new = ["A", "B", "Z","C","D"]

In [ ]:
x_transform_woe = optb.transform(x_new, metric="woe")
pd.DataFrame({'grade': x_new, "WoE": x_transform_woe})

In [ ]:
x_transform_event_rate = optb.transform(x_new, metric="event_rate")
pd.DataFrame({'grade': x_new, "Event rate": x_transform_event_rate})

In [ ]:
x_transform_bins = optb.transform(x_new, metric="bins")
pd.DataFrame({'grade': x_new, "Bin": x_transform_bins})

In [ ]:
x_transform_woe = optb.transform(x_new, metric="indices")
pd.DataFrame({'grade': x_new, "indices": x_transform_woe})

In [ ]:
binning_table.plot(metric="event_rate",style="bin",show_bin_labels=True)

In [581]:
###numerical variables
#original_array.reshape(-1, 1)
special_codes = [-999]
#user_splits = sorted([value for value in card[card['ind']=='train']['emp_length_int'].unique() if value != -999])
#print(user_splits)


#user_splits_fixed=len(user_splits) * [True]
#print(user_splits_fixed)

In [553]:
optb = OptimalBinning(name='emp_length_int', dtype="numerical", solver="mip",user_splits=user_splits,
                      user_splits_fixed=user_splits_fixed,special_codes=special_codes)

In [ ]:
optb.fit(card[card['ind']=='train']['emp_length_int'].values, 1-card[card['ind']=='train']['good_bad'].values,sample_weight=card[card['ind']=='train']['wgt'])

In [ ]:
binning_table = optb.binning_table
binning_table.build()


In [ ]:
binning_table.plot(metric="event_rate")

In [ ]:
df_tmp

In [583]:
optb = OptimalBinning(name='dti', dtype="numerical", solver="mip",special_codes=special_codes)

In [ ]:
optb.fit(card[card['ind']=='train']['dti'].values, 1-card[card['ind']=='train']['good_bad'].values,sample_weight=card[card['ind']=='train']['wgt'])

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:
plot_by_woe(df_tmp,45)

In [ ]:
df_tmp

In [619]:

import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

In [620]:

data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)

In [621]:
variable = "mean texture"
x = df[variable].values
y = data.target

In [ ]:
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")
optb.fit(x, y)

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:

optb = OptimalBinning(name=variable, dtype="numerical", solver="cp",
                      monotonic_trend="descending")
optb.fit(x, y)

In [ ]:
binning_table = optb.binning_table
binning_table.build()

In [ ]:
binning_table.plot(metric="event_rate",show_bin_labels=True)

In [ ]:

optb = OptimalBinning(name=variable, dtype="numerical", solver="cp",
                      monotonic_trend="descending", gamma=0.5)
optb.fit(x, y)

In [ ]:

binning_table = optb.binning_table
binning_table.build()

In [ ]:
binning_table.plot(metric="event_rate")